### Libraries

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
#import libraries
import pandas as pd
import matplotlib.pyplot as plt

import math, random, time, warnings, gc
from collections import deque
import numpy as np


##Costume Library to manipluate data and add indicators
from Libraries.Utils import DataHandler
from IPython.display import Markdown, display

##Artificial Intelligence  Library
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Activation, Dropout, BatchNormalization
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.losses import mean_squared_logarithmic_error  #logcosh,  #mean_squared_error, 
from tensorflow.keras.backend import clear_session
import tensorflow as tf

import plotly as py
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import Scatter
py.offline.init_notebook_mode()

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
def printmd(string):
    display(Markdown(string))

### Create And Compile Model

In [ ]:
def createAIModel():
    printmd("<h3><i><u>AI Creation, Setup and Training</u></i></h3>")
    print("Creating AI Model")
    #Create model structure 
    clear_session()
    #Create sequential model
    model = Sequential()
    #Add layers to NN
    model.add(LSTM(100, input_shape=(X_train.shape[1:]), return_sequences=True ))
    model.add(Dropout(0.25))
    model.add(LSTM(200, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(200, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(50))
    model.add(Dropout(0.25))
    
    model.add(Dense(3, activation = "softmax"))
    return model

In [ ]:
def compileAndOptimizeModel(model):
    print("Optimize And Compile Model")
    #Optimize (set learning rate) and complie model 
    optimizer= Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    #print (model.summary())
    return model

In [ ]:
def createCallBacks():
    print("Adding Call Backs")
    #Callback functions 
    #path_checkpoint = 'models/RNN_Final-{epoch:02d}-{val_acc:.3f}.model'
    #callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
    #                                      monitor='val_acc',
    #                                      verbose=1,
    #                                      save_best_only=True,
    #                                      mode = "max")

    #callback_tensorboard = TensorBoard(log_dir='./TensorBoard_logs/',
    #                                   histogram_freq=0,
    #                                   write_graph=False)


    callback_early_stopping = EarlyStopping(monitor='val_loss',
                                            patience=2, verbose=1)

    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           min_lr=1e-6,
                                           patience=1,
                                           verbose=1)
    callbacks = [callback_early_stopping,
                 callback_reduce_lr]
    return callbacks

### Train Models

In [ ]:
def trainModel(market, model, callbacks, train_x, train_y, validation_x, validation_y):
    # Train model 
    history = model.fit(
        train_x, train_y,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(validation_x, validation_y),
        callbacks=callbacks
    )
    # Save model
    #model.save("models/"+timeFrame+"/{}".format(market))
    return history, model

In [ ]:
dataHandler = DataHandler()

In [ ]:
##Settings
timeFrame='15Min'
markets=['GBPUSD','EURUSD','USDCHF','USDJPY']
trade_Lenght=1
n=1

##Batch variables
batch_size=64
minutes_considered = 100

##Training split
testSplitPCT = 0.05

#Model
epochs = 1

pair = 'EURUSD'

In [ ]:
#Get Data And Add Labels
stockData = dataHandler.getFullData(pair, timeFrame)
print (len(stockData))
stockData = dataHandler.addAllIndicators(stockData)
print (len(stockData))
stockData = dataHandler.addAllTheLabels(stockData, n)
dataHandler.printDirectionAmount(stockData)

x, y = dataHandler.preprocess_df(stockData, pair, timeFrame)
X_train, X_test, y_train, y_test = dataHandler.splitDataFrame(x, y, 0.05)
X_train, y_train = dataHandler.reshapeData(X_train, y_train)
X_test, y_test = dataHandler.reshapeData(X_test, y_test)

#SetUp AI Model
model = createAIModel()
model = compileAndOptimizeModel(model)
modelcallbacks = createCallBacks()

#Train Model
hist, model = trainModel(pair, model, modelcallbacks, X_train, y_train, X_test, y_test)

model.save("Models/"+pair+"_"+timeFrame+".h5")

### Test

In [ ]:
dataHandler = DataHandler()

In [ ]:
pair = 'EURUSD'
timeFrame='1Min'
start = time.time()
stockData = dataHandler.getFullData(pair, timeFrame)
end = time.time()
print(end - start)

print(len(stockData))

In [ ]:
stockData

In [ ]:
start = time.time()
stockData = dataHandler.addAllIndicators(stockData)
end = time.time()
print(end - start)

print (len(stockData))

In [ ]:
stockData